# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Описание данных.  
  
1. сalls — количество звонков
2. minutes — суммарная длительность звонков в минутах
3. messages — количество sms-сообщений
4. mb_used — израсходованный интернет-трафик в Мб
5. is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

pd.set_option('display.max_columns', None)

## Откройте и изучите файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(20)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.duplicated().sum()

0

Данные не нуждаются в предобработке. Пустых значений нет, дубликатов нет, все столбцы имеют правильный тип данных. 

## Разбейте данные на выборки

In [6]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=12345)

Разобьем данные на 2 выборки сначала. Одну для обучния, вторую разделим на валидационную и тестовую.

In [7]:
df_valid, df_test = train_test_split(df_test, test_size=0.5, random_state=12345)

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 3027 to 482
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [9]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1386 to 3197
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 160 to 2313
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


Разбили данные на три выборки: обучающую, валидационную и тестовую в соотношении 3:1:1 соответственно.

## Исследуйте модели

Для начала исследуем модель решающего дерева.

In [11]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [12]:
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    print('max_depth = ', depth, ': ', end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth =  1 : 0.7542768273716952
max_depth =  2 : 0.7822706065318819
max_depth =  3 : 0.7853810264385692
max_depth =  4 : 0.7791601866251944
max_depth =  5 : 0.7791601866251944


Наибольшее качество наблюдается при глубине дерева 3, а именно 0.785. Наименьшее при глубине 1 - 0.754.

Теперь исследуем модель случайный лес.

In [13]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
print("Accuracy лучшей модели на валидационной выборке:", best_accuracy, "Количество деревьев:",
      best_est, "Максимальная глубина:", best_depth)        

Accuracy лучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 8


Accuracy у случайного леса получилась выше(как и ожидалось), чем у решающего дерева.

Теперь рассмотрим логистическую регрессию.

In [14]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_log.fit(features_train, target_train)
predictions_valid_log = model_log.predict(features_valid)
print("Accuracy логистической регрессии:", accuracy_score(target_valid, predictions_valid_log))

Accuracy логистической регрессии: 0.7107309486780715


Мы изучили 3 модели и можем сделать вывод о том, что лучшей accuracy обладает модель случайного леса с количеством дереьев 40 и максимальной глубиной 8. Худшая accuracy у логистической регресии, но и работает она намного быстрее других. Для тестовых данных будем использовать лучшую модель случайного леса.

## Проверьте модель на тестовой выборке

In [15]:
features = pd.concat([features_valid, features_train])
target = pd.concat([target_valid, target_train])
best_model.fit(features, target)

RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)

In [16]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [17]:
predictions_test = best_model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
print("Качество выбранной модели случайного леса для тестовой выборки равно:", accuracy_test)

Качество выбранной модели случайного леса для тестовой выборки равно: 0.7993779160186625


## (бонус) Проверьте модели на адекватность

Для проверки на адекватность используем DummyClassifier. Это более простая модель по сравнению с теми, что мы использовали. Сравним его результаты с результатами нашей модели.

In [18]:
model_dummy = DummyClassifier()
model_dummy.fit(features_train, target_train)
predictions_dummy = model_dummy.predict(features_test)
print("Качество данной модели получилось", accuracy_score(target_test, predictions_dummy))

Качество данной модели получилось 0.6842923794712286


Как мы видим качество данной модели получилось ниже, чем любое другое, которое мы получали при использовании других моделей с разными гиперпараметрами.